In [ ]:
# 필요한 모듈을 설치하기 위한 pip 명령어
!pip install pandas numpy sentence-transformers scikit-learn

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


## SentenceBERT MODEL 불러오기

In [3]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentence = ["안녕하세요?", " 한국어 문장 임베딩을 위한 버트 모델입니다."]
sentence_embeddings = model.encode(sentence)

print(sentence_embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.37510458 -0.77338415  0.5927711  ...  0.5792351   0.32683465
  -0.65089613]
 [-0.0936174  -0.1819152  -0.19230822 ... -0.03165796  0.30412537
  -0.26793632]]


## 데이터 전처리


In [5]:
df = pd.read_csv('/content/drive/MyDrive/1조_감성대화/emotion.csv')
df

,Unnamed: 0,감정_대분류,사람문장1,시스템문장1,사람문장2,시스템문장2
0,0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.
1,1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.
2,2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.
3,3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.
4,4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...
...,...,...,...,...,...,...
58266,58266,당황,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,같은 시기에 결혼하는 친구들끼리 상황이 다르다 보니 마음이 좋지 않으시겠어요.,우리만 뒤처지는 것 같고. 그래도 열심히 살다 보면 우리도 집을 살 수 있겠지?,우리 부부만 뒤처지는 것 같아 속상하지만 열심히 살다 보면 집을 살 수 있다고 믿으...
58267,58267,당황,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,기분이 좋지 않으신 이유를 알고 계시나요?,나도 결혼했지만 아이도 생기지 않고 그 정도로 행복하지 않거든. 친구보다 못사는 것...,친구분과 비교하게 되어 기분이 좋지 않으셨군요. 어떻게 하면 기분이 풀릴 수 있을까요?
58268,58268,당황,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,돈을 모으는 것으로 자괴감이 드시는군요.,요새 집값이 너무 올라서 한숨만 나와.,집값이 올라 한숨만 나오시는군요.
58269,58269,당황,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,먼저 결혼한 친구가 부럽군요.,맞아. 그 친구와 비교하게 되니 자존감이 낮아지는 기분이야.,비교로 자존감이 낮아지는군요.


In [9]:
df = df.drop(['Unnamed: 0'], axis=1)
df

,감정_대분류,사람문장1,시스템문장1,사람문장2,시스템문장2
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...
...,...,...,...,...,...
58266,당황,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,같은 시기에 결혼하는 친구들끼리 상황이 다르다 보니 마음이 좋지 않으시겠어요.,우리만 뒤처지는 것 같고. 그래도 열심히 살다 보면 우리도 집을 살 수 있겠지?,우리 부부만 뒤처지는 것 같아 속상하지만 열심히 살다 보면 집을 살 수 있다고 믿으...
58267,당황,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,기분이 좋지 않으신 이유를 알고 계시나요?,나도 결혼했지만 아이도 생기지 않고 그 정도로 행복하지 않거든. 친구보다 못사는 것...,친구분과 비교하게 되어 기분이 좋지 않으셨군요. 어떻게 하면 기분이 풀릴 수 있을까요?
58268,당황,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,돈을 모으는 것으로 자괴감이 드시는군요.,요새 집값이 너무 올라서 한숨만 나와.,집값이 올라 한숨만 나오시는군요.
58269,당황,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,먼저 결혼한 친구가 부럽군요.,맞아. 그 친구와 비교하게 되니 자존감이 낮아지는 기분이야.,비교로 자존감이 낮아지는군요.


In [12]:
# prompt: 사람문장1, 사람문장 2 컬럼을 유저 라는 새로운 컬럼으로 합치고 시스템문장1, 시스템문장 2 컬럼도 챗봇이라는 컬럼으로 합치는 코드를 알려줘

# df['user'] = df['사람문장1'] + ' ' + df['사람문장2']
# df['챗봇'] = df['시스템문장1'] + ' ' + df['시스템문장2']
# df = df.drop(['사람문장1', '사람문장2', '시스템문장1', '시스템문장2'], axis=1)
df.head()



,감정_대분류,user,챗봇
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 혼자 해결하기로 했군요. 혼자서 ...
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요? 월급이 줄...
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...


In [13]:
df.loc[1, 'user']

'이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나. 최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.'

## Encode

In [14]:
model.encode(df.loc[1, 'user'])

array([ 2.59876788e-01,  4.64150123e-02,  3.14040959e-01, -8.29201043e-01,
       -3.46615314e-01, -1.66960701e-01, -4.90775295e-02,  3.51548523e-01,
       -2.21287385e-01, -3.41840386e-01,  2.74399787e-01,  1.68134451e-01,
       -4.10641640e-01,  6.26151800e-01, -5.84198952e-01,  5.37384033e-01,
        4.48040754e-01,  5.02166390e-01,  3.94737035e-01, -5.01793325e-01,
        3.02129894e-01, -4.33574110e-01, -3.49304639e-02, -2.92198849e-03,
       -1.97420433e-01, -1.52201891e-01,  1.49500489e-01,  2.31099978e-01,
       -1.89733177e-01,  4.07288969e-01,  3.56840253e-01,  1.15043037e-01,
        2.51129031e-01, -2.20795259e-01,  2.61838853e-01,  2.00924754e-01,
        5.61682224e-01, -2.14810252e-01, -6.57443047e-01,  5.43300927e-01,
        6.25198841e-01,  8.08288515e-01, -3.02890748e-01,  4.39833492e-01,
        1.65918411e-03, -6.28098905e-01,  6.33257031e-01,  5.17662108e-01,
       -7.26211548e-01,  3.53131771e-01, -4.81487989e-01, -5.11419475e-02,
        3.34222645e-01, -

In [17]:
df['embedding'] = pd.Series([[]]) * len(df)

df['embedding'] = df['user'].map(lambda x: model.encode(x))
df.head()


,감정_대분류,user,챗봇,hembedding,embedding
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 혼자 해결하기로 했군요. 혼자서 ...,[],"[-0.5094547, 0.21704662, 0.34772334, -0.542816..."
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요? 월급이 줄...,NaN,"[0.2598768, 0.046415012, 0.31404096, -0.829201..."
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,NaN,"[-0.3129794, -0.14694138, -0.3401636, 0.214500..."
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,NaN,"[-0.32910988, 0.011938299, -0.0071966792, -0.3..."
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,NaN,"[-0.03475588, -0.42747158, -0.0073645315, 0.08..."


In [32]:
# df.drop(['hembedding'], axis=1)

,감정_대분류,user,챗봇,embedding,distance
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 혼자 해결하기로 했군요. 혼자서 ...,"[-0.5094547, 0.21704662, 0.34772334, -0.542816...",0.064627
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요? 월급이 줄...,"[0.2598768, 0.046415012, 0.31404096, -0.829201...",0.112061
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,"[-0.3129794, -0.14694138, -0.3401636, 0.214500...",0.010109
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,"[-0.32910988, 0.011938299, -0.0071966792, -0.3...",0.073423
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,"[-0.03475588, -0.42747158, -0.0073645315, 0.08...",0.060529
...,...,...,...,...,...
58266,당황,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,같은 시기에 결혼하는 친구들끼리 상황이 다르다 보니 마음이 좋지 않으시겠어요. 우리...,"[-0.49812254, -0.44300562, -0.3988869, -0.6295...",0.254044
58267,당황,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아...,기분이 좋지 않으신 이유를 알고 계시나요? 친구분과 비교하게 되어 기분이 좋지 않으...,"[-0.74222946, -0.6584214, -0.6436984, 0.350332...",0.175414
58268,당황,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네. ...,돈을 모으는 것으로 자괴감이 드시는군요. 집값이 올라 한숨만 나오시는군요.,"[-0.19141023, -0.5811157, 0.67679536, -0.94189...",0.171642
58269,당황,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,먼저 결혼한 친구가 부럽군요. 비교로 자존감이 낮아지는군요.,"[-0.44731125, -0.5802056, -0.60427463, -0.1886...",0.176821


### csv 파일로 저장

In [36]:
df.to_csv('/content/drive/MyDrive/Pal-ette/embedding_emotiion.csv', index=False)

## 정확도 계산

In [29]:
text = '로또에 당첨됐어'

embedding = model.encode(text)

In [30]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

,감정_대분류,user,챗봇,hembedding,embedding,distance
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,많이 힘드시겠어요. 주위에 의논할 상대가 있나요? 혼자 해결하기로 했군요. 혼자서 ...,[],"[-0.5094547, 0.21704662, 0.34772334, -0.542816...",0.064627
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요? 월급이 줄...,NaN,"[0.2598768, 0.046415012, 0.31404096, -0.829201...",0.112061
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,NaN,"[-0.3129794, -0.14694138, -0.3401636, 0.214500...",0.010109
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,NaN,"[-0.32910988, 0.011938299, -0.0071966792, -0.3...",0.073423
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,NaN,"[-0.03475588, -0.42747158, -0.0073645315, 0.08...",0.060529


In [31]:
answer = df.loc[df['distance'].idxmax()]
answer

print('구분:', answer['감정_대분류'])
print('유사한 질문:', answer['user'])
print('챗봇 답변:', answer['챗봇'])
print('유사도:', answer['distance'])

구분: 기쁨
유사한 질문: 기분 좋은 일이 생겼어. 축하해 줘. 로또 일등에 당첨됐어.
챗봇 답변: 무슨 좋은 일이에요? 로또 당첨이 되셨군요. 정말 축하드려요.
유사도: 0.7971947193145752
